reference https://towardsdatascience.com/six-spark-exercises-to-rule-them-all-24

In [127]:
# Download from source
# import pandas as pd
# from tqdm import tqdm
# import csv
# import random
# import string
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import *

# random.seed(1999)

# letters = string.ascii_lowercase
# letters_upper = string.ascii_uppercase
# for _i in range(0, 10):
#     letters += letters

# for _i in range(0, 10):
#     letters += letters_upper


# def random_string(stringLength=10):
#     """Generate a random string of fixed length """
#     return ''.join(random.sample(letters, stringLength))


# print("Products between {} and {}".format(1, 75000000))
# product_ids = [x for x in range(1, 75000000)]
# dates = ['2020-07-01', '2020-07-02', '2020-07-03', '2020-07-04', '2020-07-05', '2020-07-06', '2020-07-07', '2020-07-08',
#          '2020-07-09', '2020-07-10']
# seller_ids = [x for x in range(1, 10)]


# #   Generate products
# products = [[0, "product_0", 22]]
# for p in tqdm(product_ids):
#     products.append([p, "product_{}".format(p), random.randint(1, 150)])
# #   Save dataframe
# df = pd.DataFrame(products)
# df.columns = ["product_id", "product_name", "price"]
# df.to_csv("products.csv", index=False)
# del df
# del products

# #   Generate sellers
# sellers = [[0, "seller_0", 2500000]]
# for s in tqdm(seller_ids):
#     sellers.append([s, "seller_{}".format(s), random.randint(12000, 2000000)])
# #   Save dataframe
# df = pd.DataFrame(sellers)
# df.columns = ["seller_id", "seller_name", "daily_target"]
# df.to_csv("sellers.csv", index=False)

# #   Generate sales
# total_rows = 500000
# prod_zero = int(total_rows * 0.95)
# prod_others = total_rows - prod_zero + 1
# df_array = [["order_id", "product_id", "seller_id", "date", "num_pieces_sold", "bill_raw_text"]]
# with open('sales.csv', 'w', newline='') as f:
#     csvwriter = csv.writer(f)
#     csvwriter.writerows(df_array)

# order_id = 0
# for i in tqdm(range(0, 40)):
#     df_array = []

#     for i in range(0, prod_zero):
#         order_id += 1
#         df_array.append([order_id, 0, 0, random.choice(dates), random.randint(1, 100), random_string(500)])

#     with open('sales.csv', 'a', newline='') as f:
#         csvwriter = csv.writer(f)
#         csvwriter.writerows(df_array)

#     df_array = []
#     for i in range(0, prod_others):
#         order_id += 1
#         df_array.append(
#             [order_id, random.choice(product_ids), random.choice(seller_ids), random.choice(dates),
#              random.randint(1, 100), random_string(500)])

#     with open('sales.csv', 'a', newline='') as f:
#         csvwriter = csv.writer(f)
#         csvwriter.writerows(df_array)

# print("Done")

# spark = SparkSession.builder \
#     .master("local") \
#     .config("spark.sql.autoBroadcastJoinThreshold", -1) \
#     .appName("Exercise1") \
#     .getOrCreate()

# products = spark.read.csv(
#     "products.csv", header=True, mode="DROPMALFORMED"
# )
# products.show()
# products.write.parquet("products_parquet", mode="overwrite")

# sales = spark.read.csv(
#     "sales.csv", header=True, mode="DROPMALFORMED"
# )
# sales.show()
# sales.repartition(200, col("product_id")).write.parquet("sales_parquet", mode="overwrite")

# sellers = spark.read.csv(
#     "sellers.csv", header=True, mode="DROPMALFORMED"
# )
# sellers.show()
# sellers.write.parquet("sellers_parquet", mode="overwrite")

In [128]:
spark

In [129]:
# spark = SparkSession.builder \
#     .master("local") \
#     .config("spark.sql.autoBroadcastJoinThreshold", -1) \
#     .config("spark.executor.memory", "500mb") \
#     .appName("Exercise1") \
#     .getOrCreate()

#### Warm-Up #1

- Find out how many orders, how many products and how many sellers are in the data.
- How many products have been sold at least once? Which is the product contained in more orders?

In [130]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [131]:
# read from parquet
product_table = spark.read.parquet

In [132]:
# read from cdv
products_table = spark.read\
                .format('csv')\
                .option('header', 'true')\
                .load('./products.csv')

In [133]:
sales_table = spark.read\
                .format('csv')\
                .option('header', 'true')\
                .load('./sales.csv')

In [134]:
sellers_table = spark.read\
                .format('csv')\
                .option('header', 'true')\
                .load('./sellers.csv')

In [135]:
#   Print the number of orders
print("Number of Orders: {}".format(sales_table.count()))

Number of Orders: 20000040


In [136]:
#   Print the number of sellers
print("Number of sellers: {}".format(sellers_table.count()))

Number of sellers: 10


In [137]:
#   Print the number of products
print("Number of products: {}".format(products_table.count()))

Number of products: 75000000


In [138]:
#   Output how many products have been actually sold at least once
print("Number of products sold at least once")
sales_table.agg(countDistinct(col("product_id")).alias("products_sold_at_least_one_count")).show()

Number of products sold at least once
+--------------------------------+
|products_sold_at_least_one_count|
+--------------------------------+
|                          993299|
+--------------------------------+



In [139]:
#   Output which is the product that has been sold in more orders
print("Product present in more orders")
sales_table.groupBy(col("product_id"))\
            .agg(count("*").alias("cnt"))\
                 .orderBy(col("cnt").desc())\
                 .limit(1)\
                 .show()

Product present in more orders
+----------+--------+
|product_id|     cnt|
+----------+--------+
|         0|19000000|
+----------+--------+



#### Warm-up #2

How many distinct products have been sold in each day?

In [140]:
sales_table.groupby(col("date"))\
            .agg(countDistinct(col("product_id"))\
                 .alias("distinct_products_sold"))\
                    .orderBy(col("distinct_products_sold").desc())\
                        .show()

+----------+----------------------+
|      date|distinct_products_sold|
+----------+----------------------+
|2020-07-04|                100294|
|2020-07-03|                100224|
|2020-07-10|                100218|
|2020-07-08|                100048|
|2020-07-05|                 99991|
|2020-07-06|                 99869|
|2020-07-09|                 99801|
|2020-07-02|                 99768|
|2020-07-01|                 99755|
|2020-07-07|                 99453|
+----------+----------------------+



#### Exercise #1

##### What is the average revenue of the orders?

In [141]:
from pyspark.sql import Row
from pyspark.sql.types import IntegerType
# revenue = price * quantity

In [142]:
# do the join the print the results
sales_table.join(products_table, sales_table["product_id"]==products_table["product_id"], "inner")\
                     .agg(avg(products_table["price"]*sales_table["num_pieces_sold"])).show()

+------------------------------+
|avg((price * num_pieces_sold))|
+------------------------------+
|            1245.9236386027228|
+------------------------------+



- The important thing to observe here is that we are NOT salting ALL the products, but only those that drive skewness (in the example we are getting the 100 most frequent products). 
- Salting the whole dataset would be problematic since the number of rows would grow linearly on the “salting factor”:

In [143]:
# Step 1 - Check and select the skewed keys 
# In this case we are retrieving the top 100 keys: these will be the only salted keys.
results = sales_table.groupby(sales_table["product_id"]).count().sort(col("count").desc()).limit(100).collect()


In [144]:
# Step 2 - What we want to do is:
#  a. Duplicate the entries that we have in the dimension table for the most common products, e.g.
#       product_0 will become: product_0-1, product_0-2, product_0-3 and so on
#  b. On the sales table, we are going to replace "product_0" with a random duplicate (e.g. some of them 
#     will be replaced with product_0-1, others with product_0-2, etc.)
# Using the new "salted" key will unskew the join

# Let's create a dataset to do the trick
REPLICATION_FACTOR = 101
l = []
replicated_products = []
for _r in results:
    replicated_products.append(_r["product_id"])
    for _rep in range(0, REPLICATION_FACTOR):
        l.append((_r["product_id"], _rep))
rdd = spark.sparkContext.parallelize(l)
replicated_df = rdd.map(lambda x: Row(product_id=x[0], replication=int(x[1])))
replicated_df = spark.createDataFrame(replicated_df)

In [145]:
# Step 3: Generate the salted key
products_table = products_table.join(broadcast(replicated_df),
                                     products_table["product_id"] == replicated_df["product_id"], "left"). \
    withColumn("salted_join_key", when(replicated_df["replication"].isNull(), products_table["product_id"]).otherwise(
    concat(replicated_df["product_id"], lit("-"), replicated_df["replication"])))

sales_table = sales_table.withColumn("salted_join_key", when(sales_table["product_id"].isin(replicated_products),
                                                             concat(sales_table["product_id"], lit("-"),
                                                                    round(rand() * (REPLICATION_FACTOR - 1), 0).cast(
                                                                        IntegerType()))).otherwise(
    sales_table["product_id"]))

In [146]:
#   Step 4: Finally let's do the join
print(sales_table.join(products_table, sales_table["salted_join_key"] == products_table["salted_join_key"],
                       "inner").
      agg(avg(products_table["price"] * sales_table["num_pieces_sold"])).show())

print("Ok")

+------------------------------+
|avg((price * num_pieces_sold))|
+------------------------------+
|            1245.9236386027228|
+------------------------------+

None
Ok


#### Exercise #2

Question number two was: “for each seller, what is the average % contribution of an order to the sellers’ daily quota?”.

In [147]:
# sales_table.join(sellers_table, sales_table["seller_id"] == sellers_table["seller_id"], "inner")\
#             .withColumn("ratio", sales_table["num_pieces_sold"]/sellers_table["daily_terget"]\
#             .groupBy(sales_table["seller_id"])\
#             .agg(avg("ratio")).show())

sales_table.join(sellers_table, sales_table["seller_id"] == sellers_table["seller_id"], "inner").withColumn(
    "ratio", sales_table["num_pieces_sold"]/sellers_table["daily_target"]
).groupBy(sales_table["seller_id"]).agg(avg("ratio")).show()

# Wrong way to do this - Skewed
# (Note that Spark will probably broadcast the table anyway, unless we forbid it throug the configuration paramters)

+---------+--------------------+
|seller_id|          avg(ratio)|
+---------+--------------------+
|        7|8.510553537464308E-5|
|        3|7.060842894390345E-4|
|        8|0.002071646546208688|
|        0|2.019736225290656E-5|
|        5|8.038980497173718E-5|
|        6|2.534518215186283...|
|        9|1.449276275189591...|
|        1|3.670188787905770...|
|        4|3.845384604576876E-5|
|        2| 2.45672194595152E-4|
+---------+--------------------+



In [148]:
# Correct way through broarcasting
sales_table.join(broadcast(sellers_table), sales_table["seller_id"] == sellers_table["seller_id"], "inner").withColumn(
    "ratio", sales_table["num_pieces_sold"]/sellers_table["daily_target"]
).groupBy(sales_table["seller_id"]).agg(avg("ratio")).show()

+---------+--------------------+
|seller_id|          avg(ratio)|
+---------+--------------------+
|        7|8.510553537463727E-5|
|        3|7.060842894390333E-4|
|        8|0.002071646546208...|
|        0|2.019736225262764E-5|
|        5|8.038980497175842E-5|
|        6|2.534518215186317...|
|        9|1.449276275189586...|
|        1|3.670188787904589E-5|
|        4|3.845384604576359E-5|
|        2|2.456721945951489E-4|
+---------+--------------------+



#### Exercise #3

Question: “Who are the second most selling and the least selling persons (sellers) for each product? Who are those for the product with product_id = 0”.

In [149]:
# Calculate the number of pieces sold by each seller for each product and sort by the pieces sold
sales_table = sales_table.groupby(col("product_id"), col("seller_id"))\
                            .agg(sum("num_pieces_sold").alias("num_pieces_sold"))

In [150]:
# Create the window functions, one will sort ascending the other one descending. Partition by the product_id
# and sort by pieces sold
from pyspark.sql import Row, Window
window_desc = Window.partitionBy(col("product_id")).orderBy(col('num_pieces_sold').desc())
window_asc = Window.partitionBy(col("product_id")).orderBy(col('num_pieces_sold').desc())


In [151]:
window_desc

In [152]:
# Create a Dense Rank (to avoid holes)
sales_table = sales_table.withColumn("rank_asc", dense_rank().over(window_asc))\
                            .withColumn("rank_desc", dense_rank().over(window_desc))

In [153]:
sales_table.show()

+----------+---------+---------------+--------+---------+
|product_id|seller_id|num_pieces_sold|rank_asc|rank_desc|
+----------+---------+---------------+--------+---------+
|  10005071|        7|           76.0|       1|        1|
|  10022361|        6|           92.0|       1|        1|
|  10027928|        4|           87.0|       1|        1|
|  10038026|        1|           52.0|       1|        1|
|  10106159|        6|          100.0|       1|        1|
|  10117814|        5|           65.0|       1|        1|
|  10124701|        3|           97.0|       1|        1|
|  10124876|        2|           72.0|       1|        1|
|  10125598|        4|           29.0|       1|        1|
|  10137910|        6|           68.0|       1|        1|
|   1014157|        7|           70.0|       1|        1|
|  10143170|        6|           55.0|       1|        1|
|  10145275|        8|           94.0|       1|        1|
|  10162724|        4|           86.0|       1|        1|
|  10179037|  

In [154]:
# Get products that only have one row OR the products in which multiple sellers sold the same amount
# (i.e. all the employees that ever sold the product, sold the same exact amount)
second_seller = sales_table.where(col("rank_desc") == 2).select(
    col("product_id").alias("second_seller_product_id"), col("seller_id").alias("second_seller_seller_id"),
    lit("Second top seller").alias("type")
)

In [155]:
single_seller.show()

+------------------------+----------------+--------------------+
|single_seller_product_id|single_seller_id|                type|
+------------------------+----------------+--------------------+
|                11627820|               5|Only seller or mu...|
|                15234937|               6|Only seller or mu...|
|                17794988|               6|Only seller or mu...|
|                 2020468|               8|Only seller or mu...|
|                20922486|               3|Only seller or mu...|
|                23472480|               2|Only seller or mu...|
|                24729156|               1|Only seller or mu...|
|                26261709|               2|Only seller or mu...|
|                29470571|               2|Only seller or mu...|
|                 3134676|               3|Only seller or mu...|
|                33178488|               3|Only seller or mu...|
|                35465703|               3|Only seller or mu...|
|                36078484

In [156]:
# Get the second top sellers
second_seller = sales_table.where(col("rank_desc")==2).select(
                col("product_id").alias("second_seller_product_id"), col("seller_id").alias(
                    "second_seller_sell_id"), 
                    lit("Second top seller").alias("type"))

In [157]:
second_seller.show()

+------------------------+---------------------+-----------------+
|second_seller_product_id|second_seller_sell_id|             type|
+------------------------+---------------------+-----------------+
|                11627820|                    5|Second top seller|
|                15234937|                    6|Second top seller|
|                17794988|                    6|Second top seller|
|                 2020468|                    8|Second top seller|
|                20922486|                    3|Second top seller|
|                23472480|                    2|Second top seller|
|                24729156|                    1|Second top seller|
|                26261709|                    2|Second top seller|
|                29470571|                    2|Second top seller|
|                 3134676|                    3|Second top seller|
|                33178488|                    3|Second top seller|
|                35465703|                    3|Second top sel

In [ ]:
# Get the least sellers and exclude those rows that are already included in the first piece
# We also exclude the "second top sellers" that are also "least sellers"
least_seller = sales_table.where(col("rank_asc") == 1).select(
    col("product_id"), col("seller_id"),
    lit("Least Seller").alias("type")
).join(single_seller, (sales_table["seller_id"] == single_seller["single_seller_seller_id"]) & (
        sales_table["product_id"] == single_seller["single_seller_product_id"]), "left_anti"). \
    join(second_seller, (sales_table["seller_id"] == second_seller["second_seller_seller_id"]) & (
        sales_table["product_id"] == second_seller["second_seller_product_id"]), "left_anti")

In [ ]:
# Union all the pieces
union_table = least_seller.select(
        col("product_id"),
        col("seller_id"),
        col("tyoe")
).union(second_seller.select(
        col("second_seller_product_id").alias("product_id"),
        col("seconc_seller_seller_id"),
        col("type")
)).union(single_seller.select(
        col("single_seller_product_id"),
        col("single_seller_seller_id").alias("seller_id"),
        col("type")
))
union_table.show()